<a href="https://colab.research.google.com/github/renan-peres/mfin-portfolio-management/blob/main/02_bond_portfolio_contruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bond Portfolio Selection
Bond selection with convexity > 1 and higher price sensitivity to changes in interest rates.

### Step 1: Import Libraries

In [1]:
# UDFs
from py.utils import load_and_filter_data, export_to_excel
from py.bond_selection import calculate_bond_price, add_bond_prices_to_df, calculate_duration_for_bonds, calculate_modified_duration, calculate_price_change_sensitivity

# Data manipulation libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Excel libraries
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import os

### Step 2: Define Parameters

#### Dates

In [2]:
# Define the date range
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day
# end_date = pd.to_datetime('2025-04-26')                # Report date
start_date = end_date - timedelta(days=5*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2020-06-07 to 2025-06-06
Time span: 1825 days (5.00 years)


#### Risk-free rate (T-bill, %)

In [3]:
# Load and process data
daily_risk_free_df = load_and_filter_data('data/datasets/daily_treasury_rates.csv', ['^IRX'], start_date, end_date)
# risk_free_rate = .0433 
risk_free_rate = daily_risk_free_df.iloc[-1, 0] / 100

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")
daily_risk_free_df.head()

Found 1 of 1 tickers in data/datasets/daily_treasury_rates.csv
Missing tickers: []
Risk-Free Rate: 0.0424 -- 13 WEEK TREASURY BILL (^IRX)


,^IRX
Date,
2020-06-08,0.153
2020-06-09,0.165
2020-06-10,0.165
2020-06-11,0.153
2020-06-12,0.150


#### Report File

In [4]:
output_file = f'reports/portfolio-{datetime.date(end_date)}.xlsx'

### Step 3: Import Bond Data

In [5]:
blackrock_etf_df = pd.read_csv('data/datasets/fundamentals_blackrock_bonds.csv')
blackrock_etf_df.head()

,Ticker,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,Product_ID,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon
0,AGG,iShares Core U.S. Aggregate Bond ETF,2.42,5.45,1.49,-0.91,1.45,3.05,"May 31, 2025","Sep 22, 2003",125321052024,239458,https://www.blackrock.com/us/individual/produc...,0.0469,8.16,0.52,0.0356
1,AGIH,iShares Inflation Hedged U.S. Aggregate Bond ETF,3.12,5.10,-,-,-,2.53,"May 31, 2025","Jun 22, 2022",2455344,328179,https://www.blackrock.com/us/individual/produc...,0.0474,7.90,0.51,NaN
2,AGRH,iShares Interest Rate Hedged U.S. Aggregate Bo...,1.63,4.70,-,-,-,5.34,"May 31, 2025","Jun 22, 2022",7735770,328180,https://www.blackrock.com/us/individual/produc...,0.0538,8.21,-0.16,NaN
3,AGZ,iShares Agency Bond ETF,2.51,5.62,2.58,0.37,1.71,2.31,"May 31, 2025","Nov 05, 2008",609234810,239457,https://www.blackrock.com/us/individual/produc...,0.0420,4.23,0.25,0.0348
4,BAIPX,iShares Short-Term TIPS Bond Index Fund,3.39,6.22,2.94,3.45,-,2.77,"May 31, 2025","Feb 16, 2016",5184981,282302,https://www.blackrock.com/us/individual/produc...,NaN,NaN,NaN,NaN


### Step 4: Filter for Convexity > 1

In [6]:
bond_funds_filtered_df = blackrock_etf_df[blackrock_etf_df['Convexity'] >= 1].sort_values(by='Convexity', ascending=False).reset_index(drop=True)

# Define all possible percentage columns
all_percentage_cols = ['YTD (%)', '1Y (%)', '3Y (%)', '5Y (%)', '10Y (%)', 'Incept (%)', 'Since Inception (%)']

# Filter to only include columns that actually exist in the DataFrame
percentage_cols = [col for col in all_percentage_cols if col in bond_funds_filtered_df.columns]

print(f"Processing the following percentage columns: {percentage_cols}")

# First, ensure all percentage columns are converted to numeric values
for col in percentage_cols:
    # Convert to numeric first, handling errors by setting them to NaN
    bond_funds_filtered_df[col] = pd.to_numeric(bond_funds_filtered_df[col], errors='coerce')

# Now perform the division safely
for col in percentage_cols:
    bond_funds_filtered_df[col] = bond_funds_filtered_df[col] / 100

# Format as percentage strings
for col in percentage_cols + ['Yield_To_Maturity', 'Weighted_Avg_Coupon']:
    # Check if column exists and only process if it does
    if col in bond_funds_filtered_df.columns:
        # Only format cells that aren't NaN
        bond_funds_filtered_df[col] = bond_funds_filtered_df[col].apply(
            lambda x: '{:.2%}'.format(x) if pd.notna(x) else x
        )

bond_tickers = bond_funds_filtered_df['Ticker'].tolist()

# Display Outputs
print(f"Number of iShares bond/fixed income funds with Convexity >= 1: {len(bond_tickers)}")
print(bond_tickers)
display(bond_funds_filtered_df.head())

Processing the following percentage columns: ['YTD (%)', '1Y (%)', '3Y (%)', '5Y (%)', '10Y (%)', 'Incept (%)']
Number of iShares bond/fixed income funds with Convexity >= 1: 13
['GOVZ', 'IBGL', 'TLT', 'ILTB', 'IBGB', 'IGLB', 'IBGA', 'TLH', 'ICVT', 'IGOV', 'LQD', 'ELQD', 'LQDI']


,Ticker,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,Product_ID,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon
0,GOVZ,iShares 25+ Year Treasury STRIPS Bond ETF,-4.68%,-7.93%,-13.07%,NaN,NaN,-16.50%,"May 31, 2025","Sep 22, 2020",265173345,315911,https://www.blackrock.com/us/individual/produc...,5.01%,27.24,7.21,0.00%
1,IBGL,iShares® iBonds® Dec 2055 Term Treasury ETF,NaN,NaN,NaN,NaN,NaN,NaN,"May 31, 2025","Mar 25, 2025",3620026,342146,https://www.blackrock.com/us/individual/produc...,4.89%,29.76,3.46,4.66%
2,TLT,iShares 20+ Year Treasury Bond ETF,0.02%,-0.72%,-6.31%,-9.62%,-1.00%,3.63%,"May 31, 2025","Jul 22, 2002",49853062956,239454,https://www.blackrock.com/us/individual/produc...,4.96%,25.77,3.40,2.88%
3,ILTB,iShares Core 10+ Year USD Bond ETF,0.70%,2.31%,-1.41%,-4.35%,1.49%,3.75%,"May 31, 2025","Dec 08, 2009",589613986,239424,https://www.blackrock.com/us/individual/produc...,5.60%,21.90,2.36,3.94%
4,IBGB,iShares® iBonds® Dec 2045 Term Treasury ETF,NaN,NaN,NaN,NaN,NaN,NaN,"May 31, 2025","Mar 25, 2025",3637712,342124,https://www.blackrock.com/us/individual/produc...,4.95%,19.89,2.32,3.32%


In [7]:
# Ensure relevant columns are numeric
bond_funds_filtered_df['Yield_To_Maturity'] = pd.to_numeric(
    bond_funds_filtered_df['Yield_To_Maturity'].str.replace('%', ''), errors='coerce'
)
bond_funds_filtered_df['Weighted_Avg_Coupon'] = pd.to_numeric(
    bond_funds_filtered_df['Weighted_Avg_Coupon'].str.replace('%', ''), errors='coerce'
)
bond_funds_filtered_df['Weighted_Avg_Maturity'] = pd.to_numeric(
    bond_funds_filtered_df['Weighted_Avg_Maturity'], errors='coerce'
)

# Fill missing values with 0
bond_funds_filtered_df = bond_funds_filtered_df.fillna(0)

# Divide the columns by 100 and reassign
bond_funds_filtered_df['Yield_To_Maturity'] = bond_funds_filtered_df['Yield_To_Maturity'] / 100
bond_funds_filtered_df['Weighted_Avg_Coupon'] = bond_funds_filtered_df['Weighted_Avg_Coupon'] / 100

# Display the DataFrame with the updated columns
display(bond_funds_filtered_df.head())

,Ticker,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,Product_ID,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon
0,GOVZ,iShares 25+ Year Treasury STRIPS Bond ETF,-4.68%,-7.93%,-13.07%,0,0,-16.50%,"May 31, 2025","Sep 22, 2020",265173345,315911,https://www.blackrock.com/us/individual/produc...,0.0501,27.24,7.21,0.0000
1,IBGL,iShares® iBonds® Dec 2055 Term Treasury ETF,0,0,0,0,0,0,"May 31, 2025","Mar 25, 2025",3620026,342146,https://www.blackrock.com/us/individual/produc...,0.0489,29.76,3.46,0.0466
2,TLT,iShares 20+ Year Treasury Bond ETF,0.02%,-0.72%,-6.31%,-9.62%,-1.00%,3.63%,"May 31, 2025","Jul 22, 2002",49853062956,239454,https://www.blackrock.com/us/individual/produc...,0.0496,25.77,3.40,0.0288
3,ILTB,iShares Core 10+ Year USD Bond ETF,0.70%,2.31%,-1.41%,-4.35%,1.49%,3.75%,"May 31, 2025","Dec 08, 2009",589613986,239424,https://www.blackrock.com/us/individual/produc...,0.0560,21.90,2.36,0.0394
4,IBGB,iShares® iBonds® Dec 2045 Term Treasury ETF,0,0,0,0,0,0,"May 31, 2025","Mar 25, 2025",3637712,342124,https://www.blackrock.com/us/individual/produc...,0.0495,19.89,2.32,0.0332


### Step 5: Import Quotes

In [8]:
bond_tickers = bond_funds_filtered_df['Ticker'].tolist()
print(bond_tickers)

['GOVZ', 'IBGL', 'TLT', 'ILTB', 'IBGB', 'IGLB', 'IBGA', 'TLH', 'ICVT', 'IGOV', 'LQD', 'ELQD', 'LQDI']


In [9]:
# Load and process data
bonds_daily_df = load_and_filter_data('data/datasets/daily_bond_quotes.csv', bond_tickers, start_date, end_date)
display(bonds_daily_df.head())

Found 10 of 13 tickers in data/datasets/daily_bond_quotes.csv
Missing tickers: ['IBGL', 'IBGB', 'IBGA']


,ELQD,GOVZ,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,,,
2020-06-08,NaN,NaN,58.48,55.92,50.30,60.18,112.47,22.18,139.39,136.96
2020-06-09,NaN,NaN,58.22,55.90,50.47,60.39,112.23,22.21,140.53,138.52
2020-06-10,NaN,NaN,58.16,56.33,50.77,61.04,112.97,22.20,142.15,140.57
2020-06-11,NaN,NaN,55.63,55.52,50.57,60.77,111.62,22.26,143.85,143.23
2020-06-12,NaN,NaN,56.72,55.81,50.43,60.88,112.16,22.11,143.02,141.85


### Step 6: Prepare Data (Drop Invalid Tickers)

#### Daily Quotes

In [10]:
# Identify columns with null values in first or last 50 rows
first_50_nulls = bonds_daily_df.head(50).isnull().any()
last_50_nulls = bonds_daily_df.tail(50).isnull().any()

# Columns to drop are those with nulls in first 50 OR last 50 rows
columns_to_drop = first_50_nulls | last_50_nulls
bad_columns = columns_to_drop[columns_to_drop].index.tolist()

print(f"Dropping {len(bad_columns)} columns with missing values in first/last 50 records: {bad_columns}")

# Drop those columns
bonds_daily_filtered_df = bonds_daily_df.loc[:, ~columns_to_drop]

# Display the cleaned dataframe
display(bonds_daily_filtered_df.head())

Dropping 2 columns with missing values in first/last 50 records: ['ELQD', 'GOVZ']


,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,
2020-06-08,58.48,55.92,50.30,60.18,112.47,22.18,139.39,136.96
2020-06-09,58.22,55.90,50.47,60.39,112.23,22.21,140.53,138.52
2020-06-10,58.16,56.33,50.77,61.04,112.97,22.20,142.15,140.57
2020-06-11,55.63,55.52,50.57,60.77,111.62,22.26,143.85,143.23
2020-06-12,56.72,55.81,50.43,60.88,112.16,22.11,143.02,141.85


#### Monthly Quotes

In [11]:
bonds_monthly_filtered_df = (bonds_daily_filtered_df.set_index(pd.to_datetime(bonds_daily_filtered_df.pop('Date')))
      if 'Date' in bonds_daily_filtered_df.columns else bonds_daily_filtered_df.copy())
bonds_monthly_filtered_df.index = pd.to_datetime(bonds_monthly_filtered_df.index)              
bonds_monthly_filtered_df = (bonds_monthly_filtered_df.resample('MS').last()
   .reset_index()
   .rename(columns={'index': 'Date'}))

bonds_monthly_filtered_df.set_index('Date', inplace=True)
display(bonds_monthly_filtered_df.head())

,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,
2020-06-01,59.46,57.22,50.36,61.71,113.99,22.55,144.24,143.27
2020-07-01,63.86,60.35,52.75,65.17,117.53,23.49,148.27,149.62
2020-08-01,70.40,58.16,52.80,62.75,115.44,23.77,142.85,142.07
2020-09-01,67.84,58.00,52.31,62.61,114.93,23.62,143.94,143.17
2020-10-01,68.06,57.35,52.41,61.50,114.34,23.75,139.90,138.32


### Step 7: Calculate Returns (Lognormal)

In [12]:
log_returns_df = np.log(bonds_daily_filtered_df / bonds_daily_filtered_df.shift(1))
log_returns_df = log_returns_df.dropna().sort_index(axis=0, ascending=True).sort_index(axis=1, ascending=True)
log_returns_df.head()

,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,
2020-06-09,-0.004456,-0.000358,0.003374,0.003483,-0.002136,0.001352,0.008145,0.011326
2020-06-10,-0.001031,0.007663,0.005927,0.010706,0.006572,-0.000450,0.011462,0.014691
2020-06-11,-0.044475,-0.014484,-0.003947,-0.004433,-0.012022,0.002699,0.011888,0.018746
2020-06-12,0.019404,0.005210,-0.002772,0.001808,0.004826,-0.006761,-0.005787,-0.009682
2020-06-15,0.012962,0.018463,0.003761,0.008342,0.014077,0.005862,0.000070,0.000493


### Step 8: Calculate Bond Metrics

#### Bond Price (PV)

In [13]:
# Apply the function to your DataFrame
bond_funds_price_df = add_bond_prices_to_df(bond_funds_filtered_df)

# Display the updated DataFrame with bond prices
bond_funds_price_df[['Ticker', 'Weighted_Avg_Maturity', 'Weighted_Avg_Coupon', 
                      'Yield_To_Maturity', 'Bond_Price']]

,Ticker,Weighted_Avg_Maturity,Weighted_Avg_Coupon,Yield_To_Maturity,Bond_Price
0,GOVZ,27.24,0.000,0.050,25.978
1,IBGL,29.76,0.047,0.049,96.413
2,TLT,25.77,0.029,0.050,69.929
3,ILTB,21.90,0.039,0.056,79.201
4,IBGB,19.89,0.033,0.050,79.522
5,IGLB,22.13,0.046,0.059,84.146
6,IBGA,19.04,0.040,0.049,88.712
7,TLH,17.13,0.032,0.048,80.603
8,ICVT,3.01,0.021,0.029,97.824
9,IGOV,9.65,0.022,0.029,94.315


#### Modified Duration (D*)

In [14]:
bond_tickers = bonds_daily_filtered_df.columns.tolist()
bond_funds_filtered_df = bond_funds_filtered_df[bond_funds_filtered_df['Ticker'].isin(bond_tickers)].reset_index(drop=True)

# Apply the function to your DataFrame
bond_funds_duration_df = calculate_duration_for_bonds(bond_funds_filtered_df, end_date)

# Display the updated DataFrame
print(bond_tickers)
bond_funds_duration_df[['Ticker', 'Weighted_Avg_Maturity', 'Weighted_Avg_Coupon', 'Yield_To_Maturity', 'Bond_Price', 'Duration (D*)', 'Convexity']]

['ICVT', 'IGLB', 'IGOV', 'ILTB', 'LQD', 'LQDI', 'TLH', 'TLT']


,Ticker,Weighted_Avg_Maturity,Weighted_Avg_Coupon,Yield_To_Maturity,Bond_Price,Duration (D*),Convexity
0,TLT,25.77,0.029,0.050,69.929,18.095,3.40
1,ILTB,21.90,0.039,0.056,79.201,14.904,2.36
2,IGLB,22.13,0.046,0.059,84.146,14.618,2.19
3,TLH,17.13,0.032,0.048,80.603,13.168,1.89
4,ICVT,3.01,0.021,0.029,97.824,2.881,1.17
5,IGOV,9.65,0.022,0.029,94.315,8.526,1.12
6,LQD,12.85,0.044,0.053,91.777,9.754,1.11
7,LQDI,12.50,0.000,0.052,52.833,11.698,1.06


#### Price Sensitivity to Changes in YTM (-1%)

In [15]:
# Apply the function to your DataFrame
bond_funds_sensitivity_df = calculate_price_change_sensitivity(bond_funds_duration_df)
bond_funds_sensitivity_df[['Ticker', 'Bond_Price', 'Duration (D*)', 'Convexity', 'Price Sensitivity to YTM (-1%)']]

,Ticker,Bond_Price,Duration (D*),Convexity,Price Sensitivity to YTM (-1%)
0,TLT,69.929,18.095,3.40,0.1810
1,ILTB,79.201,14.904,2.36,0.1490
2,IGLB,84.146,14.618,2.19,0.1462
3,TLH,80.603,13.168,1.89,0.1317
4,ICVT,97.824,2.881,1.17,0.0288
5,IGOV,94.315,8.526,1.12,0.0853
6,LQD,91.777,9.754,1.11,0.0975
7,LQDI,52.833,11.698,1.06,0.1170


### Step 9: Select the Best Bond (Highest Price Sentivity to Changes in YTM)

In [16]:
# Find the row index where Price Sensitivity to YTM (-1%) is at its maximum
best_bond_ticker = bond_funds_sensitivity_df["Price Sensitivity to YTM (-1%)"].idxmax()

# Get the maximum value for display
max_sensitivity = bond_funds_sensitivity_df["Price Sensitivity to YTM (-1%)"].max()

print(f"Best Bond Ticker: {best_bond_ticker}")
print(f"Maximum Price Sensitivity to YTM (-1%): {max_sensitivity:.6f}")
bond_funds_sensitivity_df.loc[best_bond_ticker]

Best Bond Ticker: 0
Maximum Price Sensitivity to YTM (-1%): 0.181000


Ticker                                                                          TLT
Name                                             iShares 20+ Year Treasury Bond ETF
YTD (%)                                                                       0.02%
1Y (%)                                                                       -0.72%
3Y (%)                                                                       -6.31%
5Y (%)                                                                       -9.62%
10Y (%)                                                                      -1.00%
Incept (%)                                                                    3.63%
Perf. as of                                                            May 31, 2025
Inception Date                                                         Jul 22, 2002
Net Assets                                                              49853062956
Product_ID                                                                  

### Step 10: Add Returns and Standard Deviation to DataFrame

In [17]:
# Step 1: Unpivot (melt) the log_returns_df
log_returns_long = log_returns_df.reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Log Return')

# Step 2: Ensure 'Log Return' column is numeric
log_returns_long['Log Return'] = pd.to_numeric(log_returns_long['Log Return'], errors='coerce')

# Step 3: Drop rows with NaN values in 'Log Return'
log_returns_long = log_returns_long.dropna(subset=['Log Return'])

# Step 4: Calculate statistics for each ticker
log_returns_stats = log_returns_long.groupby('Ticker').agg(
    Expected_Return=('Log Return', lambda x: ((1 + x).prod() ** (252 / len(x))) - 1),  # Geometric mean (annualized return)
    Standard_Deviation=('Log Return', lambda x: x.std() * np.sqrt(252))  # Annualized standard deviation
).reset_index()

# Step 6: Set Index
log_returns_stats.set_index('Ticker', inplace=True)

# Step 7: Merge with bond_funds_sensitivity_df
bond_funds_sensitivity_df = bond_funds_sensitivity_df.merge(
    log_returns_stats,
    on='Ticker',
    how='left'
)

# Step 7: Set Index
bond_funds_sensitivity_df.set_index('Ticker', inplace=True)

# Display the updated DataFrame
display(bond_funds_sensitivity_df)

,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,...,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon,Bond_Price,Duration (D*),Price Sensitivity to YTM (-1%),Expected_Return,Standard_Deviation
Ticker,,,,,,,,,,,,,,,,,,,,,
TLT,iShares 20+ Year Treasury Bond ETF,0.02%,-0.72%,-6.31%,-9.62%,-1.00%,3.63%,"May 31, 2025","Jul 22, 2002",49853062956,...,https://www.blackrock.com/us/individual/produc...,0.050,25.77,3.40,0.029,69.929,18.095,0.1810,-0.100221,0.162473
ILTB,iShares Core 10+ Year USD Bond ETF,0.70%,2.31%,-1.41%,-4.35%,1.49%,3.75%,"May 31, 2025","Dec 08, 2009",589613986,...,https://www.blackrock.com/us/individual/produc...,0.056,21.90,2.36,0.039,79.201,14.904,0.1490,-0.050903,0.128451
IGLB,iShares 10+ Year Investment Grade Corporate Bo...,0.64%,2.79%,0.55%,-2.32%,2.23%,4.21%,"May 31, 2025","Dec 08, 2009",2369476095,...,https://www.blackrock.com/us/individual/produc...,0.059,22.13,2.19,0.046,84.146,14.618,0.1462,-0.034004,0.127036
TLH,iShares 10-20 Year Treasury Bond ETF,1.39%,2.23%,-3.03%,-7.04%,-0.56%,2.80%,"May 31, 2025","Jan 05, 2007",11122247015,...,https://www.blackrock.com/us/individual/produc...,0.048,17.13,1.89,0.032,80.603,13.168,0.1317,-0.072730,0.128816
ICVT,iShares Convertible Bond ETF,3.15%,13.72%,8.04%,9.64%,0,9.29%,"May 31, 2025","Jun 02, 2015",2380035963,...,https://www.blackrock.com/us/individual/produc...,0.029,3.01,1.17,0.021,97.824,2.881,0.0288,0.073122,0.149670
IGOV,iShares International Treasury Bond ETF,8.70%,8.01%,-0.66%,-3.43%,-0.51%,0.21%,"May 31, 2025","Jan 21, 2009",1120598080,...,https://www.blackrock.com/us/individual/produc...,0.029,9.65,1.12,0.022,94.315,8.526,0.0853,-0.039959,0.096618
LQD,iShares iBoxx $ Investment Grade Corporate Bon...,2.28%,5.27%,2.21%,-0.51%,2.45%,4.40%,"May 31, 2025","Jul 22, 2002",30210426688,...,https://www.blackrock.com/us/individual/produc...,0.053,12.85,1.11,0.044,91.777,9.754,0.0975,-0.012760,0.087901
LQDI,iShares Inflation Hedged Corporate Bond ETF,3.09%,4.85%,2.02%,3.64%,0,4.01%,"May 31, 2025","May 08, 2018",90345339,...,https://www.blackrock.com/us/individual/produc...,0.052,12.50,1.06,0.000,52.833,11.698,0.1170,0.026933,0.086971


### Step 11: Export Data to Excel

In [18]:
# Check if best_bond_ticker is a numeric index instead of ticker name
if isinstance(best_bond_ticker, (int, np.integer)):
    # Get the actual ticker name from the dataframe
    if 'Ticker' in bond_funds_sensitivity_df.columns:
        best_bond_ticker = bond_funds_sensitivity_df.iloc[best_bond_ticker]['Ticker']
        print(f"Using ticker name: {best_bond_ticker}")
    else:
        # If Ticker is the index name
        best_bond_ticker = bond_funds_sensitivity_df.index[best_bond_ticker]
        print(f"Using ticker name from index: {best_bond_ticker}")

export_to_excel(output_file, {
    'bond': bond_funds_sensitivity_df,
    'daily_quotes': bonds_daily_filtered_df[[best_bond_ticker]],  
    'monthly_quotes': bonds_monthly_filtered_df[[best_bond_ticker]]   
})

Using ticker name from index: TLT
Updated sheet 'bond'
Successfully merged data into 'daily_quotes' sheet
Successfully merged data into 'monthly_quotes' sheet
Successfully exported all data to reports/portfolio-2025-06-06.xlsx
